In [364]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from skimage import io
import os
from torch.utils.data import DataLoader
from torch import nn
import torch.functional as f
from skimage import color

In [412]:
class IDCDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = color.rgb2gray(io.imread(img_path))
        y_label = torch.tensor(int(self.annotations.iloc[index, 1])).double()

        if self.transform:
            image = self.transform(image)

        return image.astype(float), torch.FloatTensor(y_label)

In [413]:
dataset = IDCDataset(csv_file='cancers.csv', root_dir='dset')

In [414]:
trainset, testset = torch.utils.data.random_split(dataset, [1500, 500])

In [415]:
trainloader = DataLoader(dataset=trainset, batch_size=1, shuffle=True)
validloader = DataLoader(dataset=trainset, batch_size=1, shuffle=True)

In [416]:
class Net(nn.Module):
    def __init__(self, D_in, H, D_out):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H)
        self.linear2 = nn.Linear(H, D_out)

    def forward(self, x):
        x = torch.relu(self.linear1(x))
        x = self.linear2(x)
        return x

In [417]:
model = Net(2500, 100, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
LOSS = []
for epoch in range(100):
    for i, (x,y) in enumerate(trainloader):
        yhat = model(x.view(-1, 2500))
        loss = criterion(yhat, y)
        LOSS.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()